In [1]:
import numpy as np

from tensorflow import keras
from keras.datasets import cifar10
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [4]:
num_classes = 10

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


In [5]:
# load pre-trained VGG16 model without top classification layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [6]:
# add custom classification layers on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

In [7]:
# creating final model for training
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [8]:
# freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [10]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# saving model weights
checkpoint = ModelCheckpoint('weights/weights.h5', monitor='val_loss', save_best_only=True)
# stop in case of overfitting
stopping = EarlyStopping(monitor='val_loss', patience=10)

In [12]:
model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_test, y_test),
    verbose=1,
    callbacks = [checkpoint, stopping]
)

Epoch 1/10
1563/1563 [==============================] - 79s 51ms/step - loss: 1.6324 - accuracy: 0.4445 - val_loss: 1.4188 - val_accuracy: 0.5100
Epoch 2/10
   3/1563 [..............................] - ETA: 1:05 - loss: 1.5797 - accuracy: 0.3854

/Users/dragan/Desktop/Nastava/NN/autoencoders/env/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1563/1563 [==============================] - 77s 49ms/step - loss: 1.3404 - accuracy: 0.5412 - val_loss: 1.3184 - val_accuracy: 0.5433
Epoch 3/10
1563/1563 [==============================] - 78s 50ms/step - loss: 1.2654 - accuracy: 0.5647 - val_loss: 1.2705 - val_accuracy: 0.5597
Epoch 4/10
1563/1563 [==============================] - 77s 49ms/step - loss: 1.2222 - accuracy: 0.5804 - val_loss: 1.2459 - val_accuracy: 0.5666
Epoch 5/10
1563/1563 [==============================] - 78s 50ms/step - loss: 1.1932 - accuracy: 0.5890 - val_loss: 1.2292 - val_accuracy: 0.5702
Epoch 6/10
1563/1563 [==============================] - 82s 52ms/step - loss: 1.1703 - accuracy: 0.5966 - val_loss: 1.2109 - val_accuracy: 0.5751
Epoch 7/10
1563/1563 [==============================] - 80s 51ms/step - loss: 1.1527 - accuracy: 0.6022 - val_loss: 1.1963 - val_accuracy: 0.5833
Epoch 8/10
1563/1563 [==============================] - 80s 51ms/step - loss: 1.1370 - accuracy: 0.6074 - val_loss: 1.1870 - val_accura

In [13]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 13s 41ms/step - loss: 1.1714 - accuracy: 0.5937
Test loss: 1.1714236736297607
Test accuracy: 0.5936999917030334
